<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/preprocessing/SolarRadiationCSV_to_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier werden die Solarstrahlungsdaten aus der Vorhersage von CAMS in einen Dataframe gespeichert:

**Dateiname: SolarRadiationCSV_to_DF**

Die Daten können als CSV-Datei über https://www.soda-pro.com/ heruntergeladen werden.

Log:

10.12.24 - Anpassung des Dataframes um eine aktualisierte Vorhersage.

## Import der benötigten Module und allgemeines Setup:

In [1]:
# Import der erfordelichen Module
import os
import numpy as np
import pandas as pd

In [2]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Pfad zur CSV-Datei auf Google Drive
import_folder = '/content/drive/My Drive/Colab_Notebooks/CSV_SolarRadiation/'
export_folder = '/content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/'

import_file_name = 'CAMS_solar_radiation.csv'
export_file_name = 'CAMS_Glo_CSV.csv'

## Import der Vorhersagewerte:

In [4]:
df_import = pd.read_csv(import_folder + import_file_name , sep = ';')
# Anpassen des Zeitstempels
df_import['Observation_period'] = pd.to_datetime(df_import['Observation_period'].str.slice(0, 19).str.replace('T', ' '))

In [5]:
# Zur Validierung werden die Vorhersagewerte im Dez. 24 erneut heruntergeladen:
# - zu diesem Zeitpunkt können für einzelne Koordinaten die Vorhersagen heruntergelden werden

import_folder_new = '/content/drive/My Drive/Colab_Notebooks/CSV_SolarRadiation/'
import_file_name_new = 'CAMS_solar_radiation.csv'

df_import_new = pd.read_csv(import_folder_new + import_file_name_new , sep = ';')
df_import_new['Observation_period'] = pd.to_datetime(df_import_new['Observation_period'].str.slice(0, 19).str.replace('T', ' '))

In [6]:
df_import_new.rename(columns={'Clear_sky_GHI': 'Clear_sky_GHI_new', 'Clear_sky_BHI': 'Clear_sky_BHI_new', 'GHI': 'GHI_new', 'BHI': 'BHI_new'}, inplace=True)

In [7]:
# Verbinden der Dataframes mit neuen und alten Vorhersagedaten:

df_merge = pd.merge(df_import, df_import_new, on='Observation_period', how='inner')

## Aggregieren der Vorhersagewerte:

Die Daten liegen mit einer minütlichen Auflösung vor und werden im Folgenden zu 2 minütlichen Werten aggregiert.

In [13]:
def zeilen_aggregieren(df):
    # Spalte erstellen, die zwei Zeilen eine Gruppe zuordnet
    df['group'] = (df.index // 2)

    # Gruppieren des Dataframes und anwenden der Aggregationsfunktion
    aggregationsfunktionen = {'Observation_period': 'first',  # Die erste Spalte bleibt unverändert
                             'Clear_sky_GHI': 'sum',          # Summe aller anderen Spalten
                             'Clear_sky_BHI': 'sum',
                             'GHI': 'sum',
                             'BHI': 'sum',
                              'Clear_sky_GHI_new': 'sum',          # Summe aller anderen Spalten
                             'Clear_sky_BHI_new': 'sum',
                             'GHI_new': 'sum',
                             'BHI_new': 'sum',
                             }

    zusammengefasstes_df = df.groupby('group').agg(aggregationsfunktionen).reset_index(drop=True)
    # Verschieben des Zeitstempels um eine Minute
    zusammengefasstes_df['Observation_period'] = zusammengefasstes_df['Observation_period'] + pd.to_timedelta(1, unit='m')

    return zusammengefasstes_df

In [14]:
df_2min = zeilen_aggregieren(df_merge)

## Abspeichern der Globalstrahlungs-Vorhersagewerte in einen Dataframe:

In [15]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [16]:
save_dataframe_to_drive(df_2min, export_folder, export_file_name)

Der DataFrame wurde als CAMS_Glo_CSV.csv in /content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/ auf Google Drive gespeichert.


In [17]:
df_2min

,Observation_period,Clear_sky_GHI,Clear_sky_BHI,GHI,BHI,Clear_sky_GHI_new,Clear_sky_BHI_new,GHI_new,BHI_new
0,2022-06-01 00:01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-06-01 00:03:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-06-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-06-01 00:07:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-06-01 00:09:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
329035,2023-08-31 23:51:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
329036,2023-08-31 23:53:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
329037,2023-08-31 23:55:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
329038,2023-08-31 23:57:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
